In [3]:
from tensorflow.keras.applications.vgg19 import VGG19,preprocess_input
from tensorflow.keras.layers import Flatten,Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import ImageFile
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
IMAGE_SIZE = [180,180]
gen_data = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = [.99,1.01],brightness_range = [0.8,1.2],data_format = "channels_last",fill_mode = "constant", horizontal_flip = True,validation_split=0.2)

train_data = gen_data.flow_from_directory('dataset/train',target_size = (180,180),batch_size = 64,class_mode = 'categorical',subset = 'training')
test_set = gen_data.flow_from_directory('dataset/TEST',target_size = (180,180),batch_size = 64,class_mode = 'categorical',subset = 'validation')




Found 1225 images belonging to 5 classes.
Found 304 images belonging to 5 classes.


In [4]:
VGG19 = VGG19(input_shape= IMAGE_SIZE+[3],weights = 'imagenet',include_top = False)
for i in VGG19.layers:
  i.trainable = False
x = Flatten()(VGG19.output)

80134624/80134624 [==============================] - 0s 0us/step


In [5]:
prediction = Dense(5,activation = 'softmax')(x)
model = Model(inputs = VGG19.input,outputs = prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 180, 180, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 180, 180, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 90, 90, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 90, 90, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 90, 90, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 45, 45, 128)       0     

In [6]:
model.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])
r = model.fit(train_data,validation_data = test_set,epochs = 50,steps_per_epoch = len(train_data),validation_steps = len(test_set))
loss,accuracy = model.evaluate(test_set,steps = 11,verbose = 2,use_multiprocessing=True,workers=2)
print(f'model performance on test images:\n Accuracy = {accuracy}\n Loss = {loss}')






Epoch 1/50
20/20 [==============================] - 382s 18s/step - loss: 1.1983 - accuracy: 0.5143 - val_loss: 0.8743 - val_accuracy: 0.6941
Epoch 2/50
20/20 [==============================] - 28s 1s/step - loss: 0.5787 - accuracy: 0.8204 - val_loss: 0.6132 - val_accuracy: 0.7796
Epoch 3/50
20/20 [==============================] - 27s 1s/step - loss: 0.4452 - accuracy: 0.8588 - val_loss: 0.5268 - val_accuracy: 0.8158
Epoch 4/50
20/20 [==============================] - 28s 1s/step - loss: 0.3768 - accuracy: 0.8914 - val_loss: 0.5132 - val_accuracy: 0.8158
Epoch 5/50
20/20 [==============================] - 27s 1s/step - loss: 0.3149 - accuracy: 0.9127 - val_loss: 0.5125 - val_accuracy: 0.8092
Epoch 6/50
20/20 [==============================] - 28s 1s/step - loss: 0.2900 - accuracy: 0.9200 - val_loss: 0.4723 - val_accuracy: 0.8092
Epoch 7/50
20/20 [==============================] - 27s 1s/step - loss: 0.2486 - accuracy: 0.9388 - val_loss: 0.4365 - val_accuracy: 0.8618
Epoch 8/50
20/20 [

In [7]:
model.save('weathermodel.h5')

In [10]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
model = load_model("/content/wcv.h5")

img = image.load_img("/content/drive/MyDrive/dataset/TEST/sunrise_3.jpg",target_size = (180,180))
x = image.img_to_array(img)
x = np.expand_dims(x,axis = 0)
preds = model.predict(x)
pred = np.argmax(preds,axis = 1)
index = ['cloudy','foggy','rainy','sunshine','sunrise']
res = str(index[pred[0]])
print(res)  

1/1 [==============================] - 1s 818ms/step
sunrise


In [11]:
import os
folder_path = "dataset/TEST/"
n_file = os.listdir(folder_path)
corr_count = 0
for image_file in n_file:
    path = "dataset/TEST/{}".format(image_file)
    img = image.load_img(path,target_size = (180,180))
    x = image.img_to_array(img)
    x = np.expand_dims(x,axis = 0)
    preds = model.predict(x)
    pred = np.argmax(preds,axis = 1)
    index = ['Cloud','foggy','rain','shine','sunrise']
    res = str(index[pred[0]])
    filename,ext = os.path.splitext(image_file)
    if(res in filename):
        corr_count+=1
acc = ((corr_count)/len(n_file))*100
print(acc)





1/1 [==============================] - 0s 21ms/step
70.0
